In [1]:
# install and imports
# imports
import requests
import os
import pandas as pd
import numpy as np
from pprint import pprint
import sqlite3
from sqlite3 import Error

all fsq cats: https://developer.foursquare.com/docs/api/venues/categories

all yp cats: https://api.yelp.com/v3/categories

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [2]:
FOURSQUARE_API_KEY = os.environ["FOURSQUARE_API_KEY"]

url = "https://api.foursquare.com/v3/places/search?"

headers_fsq = {
    "accept": "application/json",
    "Authorization": FOURSQUARE_API_KEY
}

params_fsq={ 'll' : "49.2827,-123.1207",
        'radius' : '1000',
        'limit' : '50',
        'fields' : 'rating,name,categories,fsq_id,location,distance'
       }

response_fsq = requests.get(url, headers=headers_fsq, params=params_fsq)

Parse through the response to get the POI details you want (rating, name, location, etc)

In [3]:
request_dict_fsq = response_fsq.json()
results_fsq = {'name' : [],
           'categories' : [],
           'fsq_id' : [],
           'location' : [],
           'distance' : [],
            'rating' : []}

for place in request_dict_fsq['results']:
    for key in results_fsq.keys():
        results_fsq[key].append(place[key])
        
results_df_fsq = pd.DataFrame.from_dict(results_fsq)

Put your parsed results into a DataFrame

In [4]:
def tidy_df(dirty_df, columns, api):
    """
    Function to tidy nested dataframes
    Currently only working on categories
    
    :params:
    dirty_df (pandas dataframe): dataframe to be cleaned
    columns (list[str]): column names for categories
    api (str): name of api to clean
    
    :return:
    cat_df (pandas dataframe): category dataframe cleaned and tidy
    
    :todo:
    Make it work without passing api or columns regardless of which api dataframe we pass
    """
    cats = dirty_df.pop('categories')
    cat_df = pd.DataFrame(columns=columns)
    cat_index=0
    for cat_l in cats:
        for cat_d in cat_l:
            if api.lower() == 'fsq':
                df = pd.DataFrame({'id':cat_d['id'], 'name':cat_d['name'],
                                   'icon': cat_d['icon']['prefix']+cat_d['icon']['suffix']},
                                  index=[cat_index])
            elif api.lower() == 'yelp':
                df = pd.DataFrame({'alias': cat_d['alias'], 'title': cat_d['title']}, index=[cat_index])
            else:
                return -1
            cat_df = pd.concat([cat_df, df], ignore_index = False)
            cat_df.rename({cat_df.index[-1]:cat_index}, inplace = True) 
        cat_index += 1
    return cat_df

In [5]:
# Clean & Tidy: Category and Location columns
cat_df_fsq = tidy_df(results_df_fsq, columns=['id','name','icon'], api='fsq')
loc_fsq = [pd.DataFrame.from_dict([d for d in results_df_fsq.pop('location')])]
 
# Add back Category and Location columns
# Make sure data is 'tidydata'
results_tidy_fsq = results_df_fsq.join(loc_fsq)
results_tidy_fsq = results_tidy_fsq.join(cat_df_fsq, how='right', lsuffix='_l', rsuffix='_r')
results_tidy_fsq = results_tidy_fsq.rename({'name_l':'name','id':'category_id','name_r':'category_name','icon':'category_icon'}, axis='columns').reset_index(drop=True)
results_tidy_fsq = results_tidy_fsq.replace(r'^\s*$', np.nan, regex=True)
results_tidy_fsq.drop(['formatted_address','neighborhood','category_icon'], axis=1, inplace=True)
results_tidy_fsq.astype('str')
results_tidy_fsq.rating.astype('float')
results_tidy_fsq[['distance','category_id']].astype('int')
results_tidy_fsq


,name,fsq_id,distance,rating,address,country,cross_street,locality,postcode,region,address_extended,category_id,category_name
0,Vancouver Art Gallery,4aa6f2c7f964a5209b4b20e3,22,8.7,750 Hornby St,CA,btwn Robson & W Georgia St.,Vancouver,V6Z 2H7,BC,NaN,10004,Art Gallery
1,Vancouver Art Gallery Gift Shop,4ac7d069f964a520e7b920e3,23,8.6,750 Hornby St,CA,btwn Robson & W Georgia St,Vancouver,V6Z 2H7,BC,NaN,17089,Gift Store
2,Nordstrom Pacific Centre,5630fe28498e1bc43a7f0bc0,93,8.5,799 Robson St,CA,btwn Howe & Granville St,Vancouver,V7Y 0A2,BC,NaN,17033,Department Store
3,Hawksworth Restaurant,4d2cce46ae3a8cfa4067bf70,116,9.1,801 Georgia St W,CA,at Howe St,Vancouver,V6C 1P7,BC,# 102,13065,Restaurant
4,Sport Chek,575b1e85498eeb3ab3def626,94,8.0,788 Robson St,CA,NaN,Vancouver,V6Z 1A1,BC,NaN,17117,Sporting Goods Retail
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,Japadog,4b5fa47ff964a52000c629e3,322,8.4,530 Robson St,CA,btwn Seymour & Richards St.,Vancouver,V6B 2B7,BC,NaN,13058,Hot Dog Joint
65,Japadog,4b5fa47ff964a52000c629e3,322,8.4,530 Robson St,CA,btwn Seymour & Richards St.,Vancouver,V6B 2B7,BC,NaN,13263,Japanese Restaurant
66,The Keg Steakhouse + Bar,5170656ae4b029a050dd8a49,324,8.4,688 Dunsmuir St,CA,NaN,Vancouver,V6B 1N3,BC,NaN,13383,Steakhouse
67,Gotham Steakhouse & Cocktail Bar,4aa7f22bf964a520214e20e3,350,9.1,615 Seymour St,CA,btwn Dunsmuir & W Georgia,Vancouver,V6B 3K3,BC,NaN,13383,Steakhouse


# Yelp

In [6]:
YELP_API_KEY = os.environ["YELP_API_KEY"]

url = "https://api.yelp.com/v3/businesses/search"

headers_yp = {
    "accept": "application/json",
    "Authorization": f'Bearer {YELP_API_KEY}'
}
params_yp = {'latitude' : "49.2827",
        'longitude' : '-123.1207',
        'radius' : '1000',
        'limit' : '50'
       }

Send a request with the same location paramaters (location, radius, etc)

In [7]:
response_yelp = requests.get(url, headers=headers_yp, params=params_yp)
print(response_yelp.status_code)

200


Parse through your result and get POI details

In [8]:
request_dict_yp = response_yelp.json()
results_yp = {'name' : [],
           'categories' : [],
           'id' : [],
           'location' : [],
           'distance' : [],
           'rating' : []
          }

for place in request_dict_yp['businesses']:
    for key in results_yp.keys():
        results_yp[key].append(place[key])
        
results_df_yp = pd.DataFrame.from_dict(results_yp)

In [9]:
columns = list(results_df_yp['categories'][0][0].keys())
cat_df_yp = tidy_df(results_df_yp, columns=columns, api='yelp')

Put your parsed results into a DataFrame

In [10]:
#Clean and Tidy Categories and Location
results_df_yp = pd.DataFrame.from_dict(results_yp)
cat_df_yp = tidy_df(results_df_yp, columns=columns, api='yelp')
loc_yp = [pd.DataFrame.from_dict([d for d in results_df_yp.pop('location')])]


# Add back Category and Location columns
# Make sure data is 'tidydata'
results_tidy_yp = results_df_yp.join(loc_yp)
results_tidy_yp = results_tidy_yp.join(cat_df_yp)
results_tidy_yp = results_tidy_yp.replace(r'^\s*$', np.nan, regex=True)
results_tidy_yp = results_tidy_yp.rename({'id':'yp_id'})
results_tidy_yp.drop('display_address', axis=1, inplace=True)
for column in results_tidy_yp:
    if column not in ['distance','rating']:
        results_tidy_yp[column] = results_tidy_yp[column].astype("string")
    else:
        results_tidy_yp[column] = results_tidy_yp[column].astype("float")


In [11]:
results_tidy_yp

,name,id,distance,rating,address1,address2,address3,city,zip_code,country,state,alias,title
0,Medina Cafe,VPqWLp9kMiZEbctCebIZUA,365.528106,4.0,780 Richards Street,<NA>,<NA>,Vancouver,V6B 3A4,CA,BC,breakfast_brunch,Breakfast & Brunch
0,Medina Cafe,VPqWLp9kMiZEbctCebIZUA,365.528106,4.0,780 Richards Street,<NA>,<NA>,Vancouver,V6B 3A4,CA,BC,mideastern,Middle Eastern
0,Medina Cafe,VPqWLp9kMiZEbctCebIZUA,365.528106,4.0,780 Richards Street,<NA>,<NA>,Vancouver,V6B 3A4,CA,BC,wine_bars,Wine Bars
1,Miku,4EV_ZcQmjAmP3pmO-_nb2A,742.520586,4.5,200 Granville Street,Suite 70,<NA>,Vancouver,V6C 1S4,CA,BC,japanese,Japanese
1,Miku,4EV_ZcQmjAmP3pmO-_nb2A,742.520586,4.5,200 Granville Street,Suite 70,<NA>,Vancouver,V6C 1S4,CA,BC,sushi,Sushi Bars
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,Hubbub,HrmXBx51bAXftEMAJy0iew,185.443950,4.0,859 Hornby Street,<NA>,<NA>,Vancouver,V6Z 1T9,CA,BC,soup,Soup
48,Boulevard,DJplkq9x17wm2V4vIT4brg,230.961551,4.0,845 Burrard Street,<NA>,<NA>,Vancouver,V6Z 2K7,CA,BC,seafood,Seafood
48,Boulevard,DJplkq9x17wm2V4vIT4brg,230.961551,4.0,845 Burrard Street,<NA>,<NA>,Vancouver,V6Z 2K7,CA,BC,lounges,Lounges
49,Viet Sub,3rymK-sN2iuN3R0crSBQVQ,329.772788,4.0,520 Robson Street,<NA>,<NA>,Vancouver,V6B,CA,BC,vietnamese,Vietnamese


# Google (stretch)

Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [12]:
# Working with permissions due to 'attempt to write a readonly database' error
# 
os.chdir('/Users/calebward/LL_mini_project_II/data/')
os.chmod('/Users/calebward/LL_mini_project_II/data/', 0o774)

def create_connection(path):
    """
    Create connection to database
    :param:
    path (str): path to database (Note: in this case just the file name)
    :return:
    connection (sqlite3.connect): connection to database
    """
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection Success")
    except Error as e:
        print(f"Error: '{e}'")

    return connection


def execute_query(connection, sql):
    """
    Run a SQL query (INSERT, ETC)
    :param:
    connection (sqlite): connection to server
    sql (str): The SQL query to execute
    :return:
    NONE
    """
    cursor = connection.cursor()
    try:
        cursor.execute(sql)
        connection.commit()
        print("Query Success")
    except Error as e:
        print(f"Error: '{e}'")

def execute_read_query(connection, sql):
    """
    Run a SQL read query (SELECT, ETC)
    :param:
    connection (sqlite): connection to server
    sql (str): The SQL query to execute
    :return:
    result (pd.Dataframe): A Pandas dataframe of your result
    """
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(sql)
        result = pd.read_sql(sql, connection)
        return result
    except Error as e:
        print(f"Error: '{e}'")
        
def write_apidata_to_db(table_name, connection, data):
    """
    Write to Database using pandas instead of SQLite3
    :param:
    tabe_name (str): name of the table
    connection (sqlite): connection to server
    data (pandas.DataFrame): the Data to write
    :return:
    None
    """
    data.to_sql(name=table_name, con=conn, if_exists='append', index=False)

In [13]:
create_fsq_table = """
CREATE TABLE IF NOT EXISTS fsq_poi (
    key INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    fsq_id VARCHAR,
    distance INTEGER,
    address VARCHAR,
    country VARCHAR,
    cross_street VARCHAR,
    formatted_address VARCHAR,
    locality VARCHAR,
    neighborhood VARCHAR,
    postcode VARCHAR,
    region VARCHAR,
    address_extended VARCHAR,
    category_id DECIMAL,
    category_name VARCHAR,
    category_icon VARCHAR,
    rating DECIMAL
);
"""

create_yelp_table = """
CREATE TABLE IF NOT EXISTS yelp_poi (
    key INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR NOT NULL,
    id VARCHAR,
    distance DECIMAL,
    rating DECIMAL,
    address1 VARCHAR,
    address2 VARCHAR,
    address3 VARCHAR,
    city VARCHAR,
    zip_code VARCHAR,
    country VARCHAR,
    state VARCHAR,
    alias VARCHAR,
    title VARCHAR
    );
"""

create_db_tables = ["""
CREATE TABLE IF NOT EXISTS category (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR UNIQUE
);
""","""
CREATE TABLE IF NOT EXISTS location (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    street VARCHAR,
    addition_address VARCHAR,
    city VARCHAR,
    country VARCHAR,
    region VARCHAR,
    postcode VARCHAR
);
""","""
CREATE TABLE IF NOT EXISTS points_of_interest (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR UNIQUE,
    yelp_rating INTEGER,
    fsq_rating DECIMAL,
    cat_id INTEGER,
    fsq_id VARCHAR,
    yelp_id VARCHAR,
    loc_id INTEGER
    );
""","""
CREATE TABLE IF NOT EXISTS points_of_interest_category (
    poi_id INTEGER UNIQUE,
    cat_id INTEGER UNIQUE
    );
"""]


In [14]:
conn = create_connection('mini_db.sqlite')
execute_query(conn, create_fsq_table)
execute_query(conn, create_yelp_table)
for table in create_db_tables:
    execute_query(conn, table)

Connection Success
Query Success
Query Success
Query Success
Query Success
Query Success
Query Success


In [15]:
#Add data to database
write_apidata_to_db('yelp_poi', conn, results_tidy_yp)
write_apidata_to_db('fsq_poi', conn, results_tidy_fsq)

In [38]:
#Add data to tables
fsq_SQL =["""
INSERT OR IGNORE INTO points_of_interest (name, fsq_rating, fsq_id)
SELECT name, rating, fsq_id
FROM fsq_poi;
""","""
INSERT OR IGNORE INTO location (id, street, addition_address, city, country, region, postcode)
SELECT fsq_id, address, address_extended, locality, country, region, postcode
FROM fsq_poi;
""","""
INSERT OR IGNORE INTO category(id, name)
SELECT fsq_id, category_name
FROM fsq_poi;
"""]
yp_SQL = ["""
INSERT OR IGNORE INTO points_of_interest (name, yelp_rating, yelp_id)
SELECT name, rating, id
FROM yelp_poi;
""","""
INSERT OR IGNORE INTO location (id, street, addition_address, city, country, region, postcode)
SELECT yelp_id, address1, address2, city, country, state, zip_code
FROM yelp_poi;
""","""
INSERT OR IGNORE INTO category(id, name)
SELECT yelp_id, alias
FROM yelp_poi;
"""]

In [39]:
for table in fsq_SQL:
    execute_query(conn, table)
    
for table in yp_SQL:
    execute_query(conn, table)

Query Success
Error: 'datatype mismatch'
Error: 'datatype mismatch'
Query Success
Error: 'no such column: yelp_id'
Error: 'no such column: yelp_id'


In [20]:
# Clean tables
del_dup ="""
DELETE FROM location
WHERE rowid > (
    SELECT MIN(rowid) FROM location l
    WHERE l.street = location.street
    AND l.addition_address = location.addition_address
);
"""

In [21]:
execute_query(conn, del_dup)

Query Success


In [22]:
# Link Tables

In [37]:
for i in execute_read_query(conn, 'SELECT * FROM points_of_interest'):
    print(i)


id
name
yelp_rating
fsq_rating
cat_id
fsq_id
yelp_id
loc_id


Get the top 10 restaurants according to their rating

In [24]:
def select_top_POI(data, n = 10, keep='all', type=None):
    """
    Find the top 10 rated points of interest from different datasets with their ratings normalized between sets.
    :params:
    data (list[pandas.DataFrames]): A list of all the Dateframe to search through
    n (int): How many POI to return
    keep (str): Determines how to deal with ties that extended the data frame beyond 'n'
                'all' show ties
                'first' show first POI when there is a tie
                'last' show last POI when there is a tie
    
    :return:
    rating_df (pandas.Dataframe): A dataframe with the name and ratings with the index as rank
    """
    rating_df = pd.DataFrame()
    for df in data:
        df['rating'] = df['rating'].apply(lambda x : (x/df['rating'].max())*10)
        rating_df = pd.concat([rating_df,df[['name','rating']]])
    if type != None:
        rating_df.category[rating_df.category == type]
        
    return rating_df.drop_duplicates().nlargest(n,'rating',keep=keep).sort_values(['rating','name'],ascending=False).reset_index(drop=True)


pd.options.display.float_format = "{:,.2f}".format

In [25]:
#SQL Query to test on working DB
top_ten_POI_SQL = """
SELECT poi.name as name, category.rating as rating
FROM poi
JOIN poi_category
ON poi.poi_id = poi_category.poi_id
JOIN category
ON category.category_id = poi_category.category_id
ORDER BY category.rating DESC
LIMIT 10;
"""
top_ten_SQL = """
SELECT * FROM fsq_poi
"""

In [29]:
execute_query(conn, top_ten_SQL)


Query Success


# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).